In [ ]:
# 1) Libraries, Project Design Input Parameters, and Calling Metocean Data - Weather Windows
import pandas as pd
import numpy as np
from datetime import timedelta

# Define variables for input parameters and file paths
# TODO: These values should be set based on the project design specifications when the cell run and promt user for their input.
GCR = None  # Great Circle Route distance in km
ACL = None  # Array Cable Installation Length in km
P = None    # Total number of Floating Offshore Wind Turbines (FOWTs)
NSS = None  # Total number of Substations

# TODO: Update these paths with the location of your weather window data files
PORT_WEATHER_DATA_PATH = '' # Path to weather window data for Staging & Integration Port, e.g. 'C:\\Users\\era5_files\\era5_port_metocean_catgories_in_8hr_intervals.xlsx'
WEA_WEATHER_DATA_PATH = ''  # Path to weather window data for Wind Energy Area (WEA), e.g. 'C:\\Users\\era5_files\\era5_wea_metocean_catgories_in_8hr_intervals.xlsx'

# Function to input project design parameters
def get_input_parameters():
    global GCR, ACL, P, NSS
    GCR = float(input("Enter the Great Circle Route distance (in km): "))
    ACL = float(input("Enter the Array Cable Installation Length (in km): "))
    P = int(input("Enter the total number of FOWTs to be deployed: "))
    NSS = int(input("Enter the total number of Substations to be deployed: "))

# Function to load weather window data from Excel files
def load_weather_data(file_path):
    return pd.read_excel(file_path)

# Getting input parameters
get_input_parameters()

# Loading weather data for Staging & Integration Port and WEA
port_weather_data = load_weather_data(PORT_WEATHER_DATA_PATH)
wea_weather_data = load_weather_data(WEA_WEATHER_DATA_PATH)

# Print the input parameters for confirmation
print(f"GCR: {GCR}, ACL: {ACL}, P: {P}, NSS: {NSS}")

In [ ]:
# 2) Task Info Cell (Base Case): Defining task information, dependencies, constraints, and number of repetitions for each task.
tasks_info = {
    'Step 1': {
        'duration': 'GCR / 4',  # to be calculated with GCR input
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'], # weather windows allowed for this task to be executed
        'repetitions': '1', # number of repititions this task will be executed for the complete project
        'location': 'WEA', # location of the task (port or WEA) for weather window selection
    },
    'Step 2': {
        'duration': '48',
        'weather_conditions': ['WC_1', 'WC_2'],
        'repetitions': 'NSS', 
        'location': 'port',  
    },
    'Step 3': {
        'duration': '36', # long integration time although it's assummed that it will be shorther by the time FOWT installations being conducted for commercial scale projects.
        'weather_conditions': ['WC_1', 'WC_2'],
        'repetitions': 'P',  
        'location': 'port', 
    },
    'Step 4': {
        'duration': '8 + (GCR / 22)',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'],
        'repetitions': 'P',
        'location': 'WEA', 
    },
    'Step 5': {
        'duration': 'ACL / 4',  
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': '1',
        'location': 'WEA', 
    },
    'Step 6': {
        'duration': '18', 
        'weather_conditions': ['WC_1', 'WC_2'],
        'repetitions': 'P',
        'location': 'port'
    },
    'Step 7': {
        'duration': '18', 
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': 'P',
        'location': 'WEA'
    },
    'Step 8': {
        'duration': 'GCR / 4.5',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'], 
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 9': {
        'duration': 'GCR / 7.5', 
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'], 
        'repetitions': 'P/2', 
        'location': 'WEA', 
    },
    'Step 10': {
        'duration': '24',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 11': {
        'duration': '36',  
        'dependencies': ['Step 9'],  
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'P/2',  
        'location': 'WEA', 
    },
    'Step 12': {
        'duration': '5 * GCR', # may need to update, the process pace may be faster or slower depending on the vessel capabilities at the time of commercial scale deployments of FOWTs and location specifications.'
        'dependencies': ['Step 1', 'Step 10'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'NSS',
        'location': 'WEA', 
    },
    'Step 13': {
        'duration': '5 * ACL', # may need to update, the process pace may be faster or slower depending on the vessel capabilities at the time of commercial scale deployments of FOWTs and location specifications.'
        'dependencies': ['Step 1', 'Step 5', 'Step 10', 'Step 11'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': '1',
        'location': 'WEA'
    },
    'Step 14': {
        'duration': '24',
        'dependencies': ['Step 12'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 15': {
        'duration': '12',
        'dependencies': ['Step 13'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'P', 
        'location': 'WEA', 
    },
}
# Note: The 'duration' in each step is a formula or a fixed value depending on the task. 
# It may require adjustment based on project specifics, technology and equipment availability and real-time conditions.
# 'weather_conditions' are defined as per the acceptable conditions for each task and their execution location.
# 'repetitions' indicate how many times each task is expected to be carried out during the project.
# 'location' specifies whether the task is at the port or the Wind Energy Area (WEA).

# The tasks_info dictionary is a critical component for the simulation, and any changes here will directly affect the simulation output.

In [137]:
# 3) Task Info Cell (Sensitivity Case 1):  # This cell adjusts the task information for Sensitivity Case 1, focusing on bottleneck steps, (Steps 3, 6, 7, 11, 15).
# It modifies durations and operational weather windows for certain tasks to facilitate sensitivity analysis.

tasks_info = {
    'Step 1': {
        'duration': 'GCR / 4', 
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'], 
        'repetitions': '1', 
        'location': 'WEA', 
    },
    'Step 2': {
        'duration': '48',
        'weather_conditions': ['WC_1', 'WC_2'],
        'repetitions': 'NSS',
        'location': 'port',  
    },
    'Step 3': {
        'duration': '30', # modified for sensitivity analysis
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'], # modified for sensitivity analysis
        'repetitions': 'P', 
        'location': 'port', 
    },
    'Step 4': {
        'duration': '8 + (GCR / 22)',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'],
        'repetitions': 'P',
        'location': 'WEA', 
    },
    'Step 5': {
        'duration': 'ACL / 4',  
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': '1',
        'location': 'WEA', 
    },
    'Step 6': {
        'duration': '16', # modified for sensitivity analysis
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'], # modified for sensitivity analysis
        'repetitions': 'P',
        'location': 'port'
    },
    'Step 7': {
        'duration': '16', # modified for sensitivity analysis
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'], # modified for sensitivity analysis
        'repetitions': 'P',
        'location': 'WEA'
    },
    'Step 8': {
        'duration': 'GCR / 4.5',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'], 
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 9': {
        'duration': 'GCR / 7.5', 
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'], 
        'repetitions': 'P/2', 
        'location': 'WEA', 
    },
    'Step 10': {
        'duration': '24',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 11': {
        'duration': '32', # modified for sensitivity analysis
        'dependencies': ['Step 9'],  
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'], # modified for sensitivity analysis
        'start_time': None,
        'repetitions': 'P/2',  
        'location': 'WEA', 
    },
    'Step 12': {
        'duration': '5 * GCR', 
        'dependencies': ['Step 1', 'Step 10'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'],
        'start_time': None,
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 13': {
        'duration': '5 * ACL', 
        'dependencies': ['Step 1', 'Step 5', 'Step 10', 'Step 11'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'],
        'start_time': None,
        'repetitions': '1',
        'location': 'WEA'
    },
    'Step 14': {
        'duration': '24',
        'dependencies': ['Step 12'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 15': {
        'duration': '10', # modified for sensitivity analysis
        'dependencies': ['Step 13'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'], # modified for sensitivity analysis
        'start_time': None,
        'repetitions': 'P', 
        'location': 'WEA', 
    },
}

In [197]:
# 4) Task Info Cell (Sensitivity Case 2): # This cell outlines task information for Sensitivity Case 2, focusing on bottleneck steps, (Steps 3, 6, 7, 11, 15).
# It modifys base case scenario bottleneck tasks' execution pace or durations of for sensitivity case 2.

tasks_info = {
    'Step 1': {
        'duration': 'GCR / 4', 
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'], 
        'repetitions': '1', 
        'location': 'WEA', 
    },
    'Step 2': {
        'duration': '48',
        'weather_conditions': ['WC_1', 'WC_2'],
        'repetitions': 'NSS', 
        'location': 'port',  
    },
    'Step 3': {
        'duration': '30', # modified for sensitivity analysis
        'weather_conditions': ['WC_1', 'WC_2'],
        'repetitions': 'P',  
        'location': 'port', 
    },
    'Step 4': {
        'duration': '8 + (GCR / 22)',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'],
        'repetitions': 'P',
        'location': 'WEA', 
    },
    'Step 5': {
        'duration': 'ACL / 4',  
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': '1',
        'location': 'WEA', 
    },
    'Step 6': {
        'duration': '16', # modified for sensitivity analysis
        'weather_conditions': ['WC_1', 'WC_2'],
        'repetitions': 'P',
        'location': 'port'
    },
    'Step 7': {
        'duration': '16', # modified for sensitivity analysis
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': 'P',
        'location': 'WEA'
    },
    'Step 8': {
        'duration': 'GCR / 4.5',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'], 
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 9': {
        'duration': 'GCR / 7.5', 
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3', 'WC_4'], 
        'repetitions': 'P/2', 
        'location': 'WEA', 
    },
    'Step 10': {
        'duration': '24',
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 11': {
        'duration': '32', # modified for sensitivity analysis
        'dependencies': ['Step 9'],  
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'P/2',  
        'location': 'WEA', 
    },
    'Step 12': {
        'duration': '5 * GCR', 
        'dependencies': ['Step 1', 'Step 10'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 13': {
        'duration': '5 * ACL', 
        'dependencies': ['Step 1', 'Step 5', 'Step 10', 'Step 11'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': '1',
        'location': 'WEA'
    },
    'Step 14': {
        'duration': '24',
        'dependencies': ['Step 12'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'NSS', 
        'location': 'WEA', 
    },
    'Step 15': {
        'duration': '10', # modified for sensitivity analysis
        'dependencies': ['Step 13'],
        'weather_conditions': ['WC_1', 'WC_2', 'WC_3'],
        'start_time': None,
        'repetitions': 'P', 
        'location': 'WEA', 
    },
}

In [ ]:
# 5) Execution Cell: Simulation Execution and Result Generation

import pandas as pd
from dateutil import parser
from datetime import timedelta 

# Function to check if weather conditions are suitable for a given task at a given start time.
def is_weather_condition_suitable(task, start_time, port_weather_data, wea_weather_data):
    # Choose the correct weather data based on the task's location (port or wind energy area (WEA)).
    weather_data = port_weather_data if task['location'] == 'port' else wea_weather_data
    # Sort the weather data by the 'Start Time' of the weather intervals.
    sorted_intervals = weather_data.sort_values('Start Time')

    # Iterate through the sorted weather intervals.
    for index, row in sorted_intervals.iterrows():
        # If the start time of the weather interval is after the task's start time, stop checking.
        if row['Start Time'] > start_time:
            break
        # If the task's start time is within a suitable weather interval, return True.
        if row['End Time'] >= start_time and row['WC_Category'] in task['weather_conditions']:
            return True
    # If no suitable interval is found, return False.
    return False

# Function to execute the simulation of tasks based on weather data and start times.
def execute_simulation(tasks_info, port_weather_data, wea_weather_data, start_time_str):
    # Dictionary to keep track of completed tasks.
    completed_tasks = {}
    # Locks for tasks based on their sequence number, if greater than 5, they start locked.
    task_locks = {task: True if int(task.split(' ')[1]) > 5 else False for task in tasks_info}

    # Initialize execution counts for steps 3, 4, 6, 7, 9, and 11 to zero.
    step_3_executions = 0
    step_4_executions = 0
    step_6_executions = 0
    step_7_executions = 0
    step_9_executed_count = 0
    step_11_executed_count = 0
    # Flags to indicate if step 2 has been executed at least once, and if steps 6 and 7 have both been executed at least once.
    step_2_executed_once = False
    step_6_7_executed_once = False

    # Parse the simulation start time from a string into a datetime object.
    current_time = parser.parse(start_time_str)
    # Store the simulation start time for reference.
    simulation_start_time = current_time

    # List to collect task execution details for analysis.
    task_data = []

    # Main simulation loop that runs until all tasks are completed the required number of times.
    while not all(completed_tasks.get(task_name, 0) >= int(eval(str(task.get('repetitions', '1')), {'NSS': NSS}, {'P': P}))
               for task_name, task in tasks_info.items()):
        # Flag to check if any tasks have been executed in the current iteration.
        tasks_executed = False

        # Before checking conditions, calculate the total number of executions for steps 3 with 6, 4 with 7, and 9 with 11.
        total_executions_3_6 = step_3_executions + step_6_executions
        total_executions_4_7 = step_4_executions + step_7_executions
        total_executions_9_11 = step_9_executed_count + step_11_executed_count

        # Iterate over each task.
        for task_name, task in tasks_info.items():
            # Get the number of repetitions for the task.
            task_repetitions = int(eval(str(task.get('repetitions', '1')), {'NSS': NSS}, {'P': P}))
            # Calculate task duration.
            task_duration = eval(task['duration'], {'GCR': GCR, 'ACL': ACL, 'P': P, 'NSS': NSS})

            # Logic for Step 3,4,6,7
            if task_name == 'Step 3':
                if task_locks['Step 3'] or step_3_executions > step_6_executions:
                    continue

            if task_name == 'Step 6':
                if task_locks['Step 6']:
                    continue

            if task_name == 'Step 4':
                if task_locks['Step 4'] or step_4_executions > step_7_executions:
                    continue

            if task_name == 'Step 7':
                if task_locks['Step 7']:
                    continue

            # Revised logic for Step 7 and 4
            if task_name in ['Step 4', 'Step 7']:
                if total_executions_4_7 >= 8:  # After first two cycles
                    if total_executions_4_7 > (total_executions_9_11 * 4):
                        continue  # Wait for Steps 9 and 11 to catch up
        
            # Revised logic for Step 9 and 11
            if task_name in ['Step 9', 'Step 11']:
                # Check if total_executions_9_11 is approaching its final execution
                if total_executions_9_11 >= (P - 1):
                    # Allow last execution if it's exactly half of total_executions_3_6
                    if total_executions_9_11 >= (total_executions_3_6 // 2):
                        continue
                else:
                    # Standard check for most of the simulation
                    if total_executions_9_11 >= (total_executions_3_6 // 2) - 1:
                        continue  # Wait for another round of Steps 3,4,6,7

            # Revised logic for Step 9 and 11
            if task_name in ['Step 9', 'Step 11']:
                # Check if total_executions_9_11 is approaching its final execution
                if total_executions_9_11 >= (P - 1):
                    # Allow last execution if it's exactly half of total_executions_4_7
                    if total_executions_9_11 >= (total_executions_4_7 // 2):
                        continue
                else:
                    # Standard check for most of the simulation
                    if total_executions_9_11 >= (total_executions_4_7 // 2) - 1:
                        continue  # Wait for another round of Steps 3,4,6,7

            # Check if the task has already been completed the required number of times or is locked.
            if completed_tasks.get(task_name, 0) >= task_repetitions or task_locks[task_name]:
                continue

            # Check if weather conditions are suitable for the task including its duration.
            if is_weather_condition_suitable(task, current_time, port_weather_data, wea_weather_data):
                task_start_time = current_time
                print(f"Executing {task_name} at {task_start_time}")

                # Update the current time by adding the task duration to it.
                current_time += timedelta(hours=task_duration)
                task_end_time = current_time

                # Update the count of completed tasks.
                completed_tasks[task_name] = completed_tasks.get(task_name, 0) + 1
                tasks_executed = True

                # Collect task details for analysis.
                task_details = {
                    'Task Name/Number': task_name,
                    'Start Date': task_start_time.strftime("%m/%d/%Y %H:%M:%S"),
                    'End Date': task_end_time.strftime("%m/%d/%Y %H:%M:%S"),
                    'Current Repetition': completed_tasks[task_name],
                    'Days Since Project Start': (task_start_time - simulation_start_time).days
                }
                task_data.append(task_details)

                # Unlocking logic for Step 13
                if 'Step 13' in task_locks and task_locks['Step 13']:
                    if step_11_executed_count >= (P / 2) and completed_tasks.get('Step 10', 0) >= 1:
                        if all(completed_tasks.get(step, 0) >= P for step in ['Step 3', 'Step 4', 'Step 6', 'Step 7']):
                            task_locks['Step 13'] = False
                            print("Step 13 unlocked")

                # Add these print statements to track the completion counts
                if task_name in ['Step 3', 'Step 4', 'Step 6', 'Step 7', 'Step 9', 'Step 10', 'Step 11']:
                    print(f"Completion Counts - Step 3: {completed_tasks.get('Step 3', 0)}, Step 4: {completed_tasks.get('Step 4', 0)}, Step 6: {completed_tasks.get('Step 6', 0)}, Step 7: {completed_tasks.get('Step 7', 0)}, Step 9: {completed_tasks.get('Step 9', 0)}, Step 10: {completed_tasks.get('Step 10', 0)}, Step 11: {completed_tasks.get('Step 11', 0)}")

                if task_name == 'Step 3':
                    step_3_executions += 1
                elif task_name == 'Step 4':
                    step_4_executions += 1
                elif task_name == 'Step 6':
                    step_6_executions += 1
                elif task_name == 'Step 7':
                    step_7_executions += 1
                elif task_name == 'Step 9':
                    step_9_executed_count += 1
                elif task_name == 'Step 11':
                    step_11_executed_count += 1
                elif task_name == 'Step 2':
                    step_2_executed_once = True

                # Update lock status based on task completion
                if task_name in ['Step 3', 'Step 4', 'Step 6', 'Step 7', 'Step 9', 'Step 11']:
                    task_locks[task_name] = True  # Lock task after execution

                # Unlocking logic for other steps
                if task_name == 'Step 3':
                    task_locks['Step 6'] = False
                elif task_name == 'Step 4':
                    task_locks['Step 7'] = False
                elif task_name in ['Step 6', 'Step 7']:
                    task_locks['Step 3'] = task_locks['Step 4'] = False
                    if step_6_executions >= 1 and step_7_executions >= 1:
                        step_6_7_executed_once = True

                # Unlocking Step 8
                if (step_2_executed_once and step_6_7_executed_once) or task_name == 'Step 8':
                    task_locks['Step 8'] = False

                if task_name == 'Step 8':
                    task_locks['Step 10'] = False

                # Step 12 unlocking and further steps
                if task_name == 'Step 10':
                    if completed_tasks.get('Step 1', 0) >= 1:
                        task_locks['Step 12'] = False
                
                # Step 12 unlocking and further steps
                if task_name == 'Step 1':
                    if completed_tasks.get('Step 10', 0) >= 1:
                        task_locks['Step 12'] = False

                if task_name == 'Step 12':
                    task_locks['Step 14'] = False

                if task_name == 'Step 9':
                    task_locks['Step 11'] = False

                if task_name == 'Step 11':
                    task_locks['Step 9'] = False

                # Unlocking logic for Step 13
                if task_name == 'Step 11' and completed_tasks.get('Step 11', 0) >= (P / 2) and completed_tasks.get('Step 10', 0) >= 1:
                    if all(completed_tasks.get(step, 0) >= P for step in ['Step 3', 'Step 4', 'Step 6', 'Step 7']):
                        task_locks['Step 13'] = False

                if task_name == 'Step 13':
                    task_locks['Step 15'] = False

                # Unlocking Step 9
                if step_6_executions >= 2 and step_7_executions >= 2 and not step_9_executed_count:
                    task_locks['Step 9'] = False

                print(f"Task {task_name} executed and finished at {task_end_time}. Lock status: {task_locks}") #Lock status: {task_locks}

        if not tasks_executed:
            current_time += timedelta(hours=1)

    # After simulation completes
    simulation_end_time = current_time
    total_duration = simulation_end_time - simulation_start_time
    print("Simulation Complete.")
    print(f"Total Simulation Duration: {total_duration.days} days, {total_duration.seconds // 3600} hours, {(total_duration.seconds // 60) % 60} minutes")

    return completed_tasks, task_data

# TODO Define years for simulation in chunks
simulation_chunks = [] #e.g. [(1972, 1981), (1982, 1991), (1992, 2001),(2002, 2011), (2012, 2021)]

for chunk in simulation_chunks:
    for year in range(chunk[0], chunk[1] + 1, 2):
        print(f"Starting simulation for the year: {year}")
        start_time_str = f"{year}/01/01 00:00:00 AM"
        completed_tasks, task_data = execute_simulation(tasks_info, port_weather_data, wea_weather_data, start_time_str)
        
        df = pd.DataFrame(task_data)
        excel_path = f'' # TODO Insert results directory for simulation data to be stored and later illustrated, e.g. 'C:\\Users\\simulation_results_directory\\simulation_data_{year}.xlsx'
        df.to_excel(excel_path, index=False)
        print(f"Results extracted to Excel file at {excel_path}")
        print(f"Simulation for {year} completed.")

In [ ]:
# 6) Resulting Graph 1: Project Completion Time and FOWTs Deployed Over the Years and Distribution of Project Completion Times

import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec

# Simulation results directory containing the Excel files
# TODO: Update 'directory' with the path where the Excel result files are located
directory = '' # e.g. 'C:\\Users\\simulation_results_directory'

# Helper function to extract the year from a file name
def extract_year(filename):
    parts = filename.split('_')
    if parts[-1].split('.')[0].isdigit():
        return int(parts[-1].split('.')[0])
    return None

# Dictionaries to hold project completion times and the number of FOWTs deployed
project_completion_times = {}
fowts_deployed_per_year = {}

# Loop through each Excel file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.xlsx'):
        year = extract_year(filename)
        if year is not None:  # Check if a valid year is found
            df = pd.read_excel(os.path.join(directory, filename))

            # Convert 'Start Date' and 'End Date' to datetime objects
            df['Start Date'] = pd.to_datetime(df['Start Date'])
            df['End Date'] = pd.to_datetime(df['End Date'])

            # Calculate the project duration for the year
            project_duration = (df['End Date'].max() - df['Start Date'].min()).days
            project_completion_times[year] = project_duration

            # Count the number of FOWTs deployed
            fowts_deployed = df[df['Task Name/Number'].str.contains('Step 15', na=False)].shape[0]
            fowts_deployed_per_year[year] = fowts_deployed

# Sort the years and extract corresponding completion times
years = sorted(project_completion_times.keys())
completion_times = [project_completion_times[year] for year in years]

# Create a figure
fig = plt.figure(figsize=(20, 10))

# Define the width ratios for the two subplots
gs = gridspec.GridSpec(1, 2, width_ratios=[12, 8])

# Create two subplots with different widths
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])

# Bar chart for project completion times per year
ax1.bar(years, completion_times, color='tab:blue', alpha=0.6, label='Completion Time')
ax1.set_xlabel('Year', fontsize=14)
ax1.set_ylabel('Project Completion Time (Days)', fontsize=14)
ax1.set_ylim([min(completion_times) - 10, max(completion_times) + 10]) # +50 for padding
ax1.yaxis.set_major_locator(ticker.MultipleLocator(20))
ax1.legend(loc='upper left', fontsize=12)
ax1.set_title('Project Completion Periods Over 50-Year', fontsize=16)

# Add data labels to each bar
for year, time in zip(years, completion_times):
    ax1.text(year, time + 1, str(time), ha='center', va='bottom', fontsize=12)

# Calculate mean, median, max, and min for completion times
mean_completion_time = np.mean(completion_times)
median_completion_time = np.median(completion_times)
max_completion_time = np.max(completion_times)
min_completion_time = np.min(completion_times)

# Add horizontal lines for mean, max, and min values
ax1.axhline(mean_completion_time, color='darkgreen', linestyle='--', linewidth=2, label='Mean')
ax1.axhline(max_completion_time, color='darkred', linestyle='-', linewidth=2, label='Max')
ax1.axhline(min_completion_time, color='purple', linestyle=':', linewidth=2, label='Min')

# Update the legend to include lines for mean, max, and min
ax1.legend(loc='upper left', fontsize=12)

# Calculate statistics for the box plot
median_completion_time = np.median(completion_times)
quartile1 = np.percentile(completion_times, 25)
quartile3 = np.percentile(completion_times, 75)
whisker_low = quartile1 - 1.5 * (quartile3 - quartile1)
whisker_high = quartile3 + 1.5 * (quartile3 - quartile1)

# Create the box plot data
box_plot_data = [whisker_low, quartile1, median_completion_time, quartile3, whisker_high]

# Plot the box plot right after the last bar
box_width = np.diff(ax1.get_xlim())[0] / (len(years) * 3)
ax1.boxplot(box_plot_data, positions=[years[-1] + 2], widths=box_width, showfliers=False, patch_artist=True)

# Set the x-axis ticks and labels
ax1.set_xticks(np.append(years, years[-1] + 2))
ax1.set_xticklabels(np.append(years, "Distribution"), rotation=45)

# Histogram for the distribution of project completion times
ax2.hist(completion_times, bins=10, color='grey', alpha=0.7, edgecolor='black')
ax2.set_xlabel('Project Completion Time (Days)', fontsize=14)
ax2.set_ylabel('Frequency', fontsize=14)
ax2.set_title('Distribution of Project Completion Periods', fontsize=16)

# Fine-tune layout
fig.tight_layout()

# Add a black frame around the entire figure
fig.patch.set_linewidth(4)  # Set the linewidth of the figure patch (border)
fig.patch.set_edgecolor('black')  # Set the edgecolor of the figure patch to black

# Add text boxes for total FOWTs deployed and each period's deployment
total_fowts_installed = sum(fowts_deployed_per_year.values())
fowts_each_period = fowts_deployed_per_year[max(years)]  # Use the latest year's FOWT count
# ax1.text(1.75, -0.1, f'Total FOWTs Deployed: {total_fowts_installed}', 
         #transform=ax1.transAxes, horizontalalignment='right', fontsize=14, color='red')
ax1.text(1.75, -0.15, f'FOWTs Deployed Each Period: {fowts_each_period}', 
         transform=ax1.transAxes, horizontalalignment='right', fontsize=14, color='red')

# Show the plot
plt.show()

In [ ]:
# 7) Resulting Graph 2 and Graph 3: Installation Task Performance and Inefficiencies 

import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as ticker

# Simulation results directory containing the Excel files
# TODO: Update 'directory' with the path where the Excel result files are located
directory = '' # e.g. 'C:\\Users\\simulation_results_directory'

# Helper function to extract the year from the filename
def extract_year(filename):
    parts = filename.split('_')
    if parts[-1].split('.')[0].isdigit():
        return int(parts[-1].split('.')[0])
    return None

# Initialize dictionaries to store data
project_completion_times = {}
fowts_deployed_per_year = {}
total_fowts_installed = 0  # Tracks the total FOWTs installed
task_durations = {}  # Accumulates total durations for each task
task_counts = {}  # Counts occurrences of each task
task_delays = {}  # Accumulates total delays for each task

# Loop through each Excel file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.xlsx'):
        year = extract_year(filename)
        if year:  # Ensure a valid year is found
            df = pd.read_excel(os.path.join(directory, filename))

            # Convert 'Start Date' and 'End Date' to datetime objects
            df['Start Date'] = pd.to_datetime(df['Start Date'])
            df['End Date'] = pd.to_datetime(df['End Date'])

            # Calculate the total project duration for the year
            project_duration = (df['End Date'].iloc[-1] - df['Start Date'].iloc[0]).days
            project_completion_times[year] = project_duration

            # Count the number of FOWTs deployed
            fowts_deployed = df[df['Task Name/Number'] == 'Step 15'].shape[0]
            fowts_deployed_per_year[year] = fowts_deployed
            total_fowts_installed += fowts_deployed

            # Calculate task durations, delays, and count occurrences
            df['Previous End Date'] = df['End Date'].shift(1).fillna(df['Start Date'])
            df['Task Duration'] = (df['End Date'] - df['Start Date']).dt.total_seconds() / 3600
            df['Total Duration'] = (df['End Date'] - df['Previous End Date']).dt.total_seconds() / 3600
            df['Delay'] = df['Total Duration'] - df['Task Duration']
            
            for index, row in df.iterrows():
                task_name = row['Task Name/Number']
                if task_name not in task_durations:
                    task_durations[task_name] = 0
                    task_counts[task_name] = 0
                    task_delays[task_name] = 0
                task_durations[task_name] += row['Task Duration']
                task_counts[task_name] += 1
                task_delays[task_name] += row['Delay']

# Define 'years' after extracting data from all files
years = sorted(project_completion_times.keys())

# Calculate average completion time and total average completion time for each task
average_completion_times = {task: (task_durations[task] / task_counts[task]) for task in task_durations}
total_average_completion_times = {task: (task_durations[task] / len(years)) for task in task_durations}

# Calculate average delay time and total average delay time for each task
average_delays = {task: (task_delays[task] / task_counts[task]) for task in task_delays}
total_average_delays = {task: (task_delays[task] / len(years)) for task in task_delays}

# Sort tasks by their numerical suffix for consistent ordering
sorted_tasks = sorted(task_durations.keys(), key=lambda x: int(x.split(' ')[1]) if x.split(' ')[1].isdigit() else float('inf'))

# Begin plotting
fig = plt.figure(figsize=(20, 30))  # Create a figure
fig.patch.set_linewidth(4)  # Set linewidth of figure border
fig.patch.set_edgecolor('black')  # Set edgecolor of figure border

# Graph 2: Average Completion Time of Each Task Over the Years
ax1 = fig.add_subplot(211)  # First subplot in a 2x1 grid
bar_width = 0.35  # Bar width
offset = np.arange(len(sorted_tasks))  # Bar offsets
colors = plt.cm.viridis(np.linspace(0, 1, len(sorted_tasks)))  # Color scheme

# Plot average completion times
for idx, task in enumerate(sorted_tasks):
    ax1.bar(offset[idx] - bar_width/2, average_completion_times[task], bar_width, color=colors[idx], label=task)

# Twin axis for total average completion times
ax2 = ax1.twinx()
for idx, task in enumerate(sorted_tasks):
    ax2.bar(offset[idx] + bar_width/2, total_average_completion_times[task], bar_width, color=colors[idx], alpha=0.3)

# Set labels and titles
ax1.set_xlabel('Task', fontsize=14)
ax1.set_ylabel('Average Completion Time (Hours)', fontsize=14)
ax1.set_title('Average and Total Completion Times of Tasks', fontsize=16)
ax1.set_xticks(offset)
ax1.set_xticklabels(sorted_tasks, rotation=45, fontsize=14)
ax2.set_ylabel('Total Completion Time (Hours)', fontsize=14)

# Adjust legend and layout
ax1.legend(title='Tasks', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout(rect=[0, 0.5, 1, 1])  # Adjust layout for top plot

# Graph 3: Average Delay Times for Each Task Over the Years
ax3 = fig.add_subplot(212)  # Second subplot in a 2x1 grid

# Plot average delays
for idx, task in enumerate(sorted_tasks):
    ax3.bar(offset[idx] - bar_width/2, average_delays[task], bar_width, color=colors[idx], label=task)

# Twin axis for total average delays
ax4 = ax3.twinx()
for idx, task in enumerate(sorted_tasks):
    ax4.bar(offset[idx] + bar_width/2, total_average_delays[task], bar_width, color=colors[idx], alpha=0.3)

# Set labels and titles
ax3.set_xlabel('Task', fontsize=14)
ax3.set_ylabel('Average Delay Time (Hours)', fontsize=14)
ax3.set_title('Average and Total Delay Times of Tasks', fontsize=16)
ax3.set_xticks(offset)
ax3.set_xticklabels(sorted_tasks, rotation=45, fontsize=14)
ax4.set_ylabel('Total Delay Time (Hours)', fontsize=14)

# Adjust legend and layout
ax3.legend(title='Tasks', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout(rect=[0, 0, 1, 0.5])  # Adjust layout for bottom plot

plt.show()